# 0.0 Imports

In [2]:
import os

import pandas  as  pd
import numpy   as  np 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## 0.1 Helper Functions

## 0.2 Loading data

In [5]:
path = "../data/"
df_raw = pd.read_csv(path + "/raw/house_data.csv", sep = ",")

In [6]:
df_raw.shape

(10692, 13)

In [7]:
df_raw.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973
2,Porto Alegre,80,1,1,1,6,acept,not furnished,1000,2800,0,41,3841
3,Porto Alegre,51,2,1,0,2,acept,not furnished,270,1112,22,17,1421
4,São Paulo,25,1,1,0,1,not acept,not furnished,0,800,25,11,836


In [9]:
# Will not use columns fire insurance (R$) and total (R$)
df_raw.drop(columns= ["fire insurance (R$)", "total (R$)"], inplace = True)
df_raw.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$)
0,São Paulo,70,2,1,1,7,acept,furnished,2065,3300,211
1,São Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750
2,Porto Alegre,80,1,1,1,6,acept,not furnished,1000,2800,0
3,Porto Alegre,51,2,1,0,2,acept,not furnished,270,1112,22
4,São Paulo,25,1,1,0,1,not acept,not furnished,0,800,25


In [10]:
df1 = df_raw

# 1.0 Data Description

In [11]:
df1.shape

(10692, 11)

In [12]:
df1.isnull().sum().sort_values( ascending = True)

city                 0
area                 0
rooms                0
bathroom             0
parking spaces       0
floor                0
animal               0
furniture            0
hoa (R$)             0
rent amount (R$)     0
property tax (R$)    0
dtype: int64

In [13]:
df1.dtypes

city                 object
area                  int64
rooms                 int64
bathroom              int64
parking spaces        int64
floor                object
animal               object
furniture            object
hoa (R$)              int64
rent amount (R$)      int64
property tax (R$)     int64
dtype: object

In [16]:
df1.city.unique()

array(['São Paulo', 'Porto Alegre', 'Rio de Janeiro', 'Campinas',
       'Belo Horizonte'], dtype=object)

# 2.0 Feature Engineering

# 3.0 Data Filtering

# 4.0 EDA

# 5.0 Data Preparation

# 6.0 Feature Selection

# 7.0 Machine Learning Modelling

# 8.0 Hyperparameter Fine Tuning

# 9.0 Translation and Interpretration Error